# Exploring NASA's Planetary Exploration Budgets Through SQL Analysis. 
## By: Nawail Khan - 043, Maryam Nasir - 049, Hiba Imran - 002.

## Task 1: What is the total cost of all planetary missions over all time?

A good first step in any budgetary analysis is to determine how much money has been spent in total.

For this, we need the `mission_budgets` table. Each row represents the cost in a fiscal year, of one aspect of a project for one mission. There are 5 columns:

- `mission`: The name of the mission.
- `fiscal_year`: The year, for accounting purposes.
- `cost_type`: Fine-grained aspect of the project, e.g., "Spacecraft".
- `cost_group`: Broader aspect of the project, e.g., "Development/Implementation".
- `cost_MUSD`: Cost in million US dollars.


In [10]:
-- Calculate the total cost of all missions over all time
SELECT SUM("cost_MUSD") AS total_cost_MUSD
	FROM mission_budgets

,total_cost_musd
0,41406.554506


## Task 2: What happens when you adjust for inflation?

Due to inflation, costs in the 1960s are not directly equivalent to those in the 2020s. We need to adjust for inflation in order to get a cost in current currency.

Correction factors are available in the `inflation` table. Each row represents an inflation adjustment for a time period relative to today. ther are two columns.

- `fiscal_year`: The year, for accounting purposes. Note that in 1976, inflation was especially high, so two values are provided. "1976" represents the start of the year, and "1976TQ" represents the third quarter onwards.
- `inflation_adjustment`: Multiply currency values from the past time by this number to get current currency values.

In [7]:
SELECT 
    SUM("cost_MUSD" * 
        (CASE 
            WHEN fiscal_year = '2020' THEN inflation_adjustment
            ELSE 1  -- No adjustment needed for other years
         END)
    ) AS adjusted_total_cost_MUSD
FROM 
    mission_budgets
LEFT JOIN 
    inflation USING (fiscal_year)

,adjusted_total_cost_musd
0,41447.393306


## Task 3: Which was the most expensive mission?

According to the analysis The Europa Clipper is NASA's most expensive mission.
Why? 
Because The Europa Clipper is NASA's big-ticket mission because it's heading to Jupiter's moon, Europa, which might have a hidden ocean beneath its icy surface—potentially home to alien life! But getting there and studying it up close isn't cheap.

In [1]:
-- The mission with the highest total cost
SELECT mission, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    GROUP BY mission
    LIMIT 1

,mission,adjusted_total_cost_musd
0,Europa Clipper,5283.8103


## Task 4: How much was spent each year?

Understanding how budgets change over time is also important. NASA has not been immune to the wax and wane of the US economy over the decades, nor the change in political enthusiasm for planetary exploration. More generally, to understand your current budget, it's usually helpful to know the context of how much budget you or your predecessors had in previous years.

In [4]:
-- Calculate the inflation adjusted total cost per year
SELECT fiscal_year, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    GROUP BY fiscal_year
    ORDER BY fiscal_year

,fiscal_year,adjusted_total_cost_musd
0,1960,261.139746
1,1961,674.849760
2,1962,1624.488450
3,1963,2268.368675
4,1964,2412.935820
...,...,...
72,2031,149.605633
73,2032,149.590728
74,2033,149.593212
75,2034,128.050000


In [5]:
# Draw a bar plot of total cost vs. year
import plotly.express as px

px.bar(total_cost_by_year, x="fiscal_year", y="adjusted_total_cost_musd")

## Interpretation of the Bar plot:

### Peaks in Spending:
Peaks indicate significant spending on major projects or missions, such as the launch of new spacecraft or large-scale scientific missions like lunar missions or Mars rover landings.
### Troughs in Spending: 
Troughs represent reduced spending periods, possibly due to fewer missions or budget cuts
### Long-Term Trends: 
Long-term trends reveal changes in investment in space exploration over time. These trends reflect NASA's evolving priorities and strategic focus over time.
### Fluctuations: 
Fluctuations reflect responses to policy changes, strategic shifts, or economic conditions
### Recent Trends: 
Recent trends indicate current investment patterns, signaling shifts in priorities or budgetary constraints.

## Task 5: How much was spent on each destination?

The 1960s was famous for the "Space Race" between the United States and Russia to be the first to get a man on the moon. In the time since then, attention has also been turned to the other planets in the solar system and to small bodies like asteroids and comets.

To analyze spending by destination, we'll need to look at the `mission_details` table. Each row represenats one mission. There are 4 columns:

- `mission`: The name of the mission.
- `mission_full_name`: The formal name of the mission.
- `destination`: The place in the solar system that the mission is exploring.
- `program`: The name of the NASA program that the mission is part of.

### Instructions

- Join the budgets to the mission details on the `mission` column.
- Calculate the total cost by destination.

In [1]:
-- Calculate the inflation adjusted total cost per destination
SELECT destination, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    LEFT JOIN mission_details
    	USING(mission)
    GROUP BY destination
    ORDER BY adjusted_total_cost_MUSD DESC

,destination,adjusted_total_cost_musd
0,Mars,31725.348594
1,Outer Planets,21827.564786
2,The Moon,12982.887263
3,Small Bodies,7365.860162
4,Venus,4796.703011
5,Mercury,1519.605957
6,Earth-Sun L1,427.220100
7,The Sun,347.404110


## Task 6: How did spending by destination change over time?

### Instructions

- Join the budgets, inflation adjustment and details tables as before.
- Group the budgets by fiscal year and destination.
- Calculate the total adjusted cost for each group.
- Store the data frame as `total_cost_by_year_and_destination`.

In [7]:
-- Calculate the inflation adjusted total cost per year per destination
SELECT fiscal_year, destination, SUM("cost_MUSD" * inflation_adjustment) AS adjusted_total_cost_MUSD
	FROM mission_budgets
    LEFT JOIN inflation
    	USING(fiscal_year)
    LEFT JOIN mission_details
    	USING(mission)
    GROUP BY fiscal_year, destination
    ORDER BY fiscal_year, destination

,fiscal_year,destination,adjusted_total_cost_musd
0,1960,The Moon,261.139746
1,1961,The Moon,674.849760
2,1962,The Moon,1248.174750
3,1962,Venus,376.313700
4,1963,Mars,382.522200
...,...,...,...
277,2032,Small Bodies,21.540728
278,2033,Outer Planets,128.050000
279,2033,Small Bodies,21.543212
280,2034,Outer Planets,128.050000


In [9]:
# Draw a bar plot of total cost vs. year, colored by destination
import plotly.express as px

fig = px.bar(total_cost_by_year_and_destination, x="fiscal_year", y="adjusted_total_cost_musd", color="destination")
fig.update_layout(xaxis={"categoryorder": "category ascending"})
fig.show()

## Interpretations of the Bar plot:

### The Moon Missions: 
There was a period with significant investment in lunar missions, which might be related to the Apollo era in the late 1960s and early 1970s. This is characterized by the tall blue bars. After that, the investment drops off, which coincides with the end of the Apollo program.
### Mars Missions: 
Mars appears to have consistent investment over many years, with fluctuations. This suggests a sustained interest in Mars exploration, possibly with multiple missions such as rovers, orbiters, and landers.
### Outer Planets Missions: 
The tall yellow bars indicate substantial investments at certain intervals. These could represent flagship missions to the outer planets, such as the Galileo or Cassini missions.
### Venus Missions: 
The green bars represent missions to Venus, which seem to occur less frequently and with less investment compared to Mars or the outer planets.
### Mercury, The Sun, Small Bodies, and Earth-Sun L1 Missions: 
These categories have smaller and less frequent investments, shown by the shorter bars of various colors. This suggests that while these destinations are of interest, they may not be as high of a priority or require less budget compared to missions to the Moon, Mars, or outer planets.
### Trends Over Time: 
Overall, there seems to be a diversification of mission destinations over time, moving from a heavy focus on the Moon during the early years to a more varied spread across different solar system destinations in the later years.
### Recent Years: 
In the most recent years on the right, there is a notable presence of missions to Mars and the outer planets, indicating continued or renewed interest in these destinations.